### 1 - Import test data

In [38]:
import pandas as pd
import pandas_ta as ta
import sys
sys.path.append('../')

df = pd.read_csv('../DB/5m/INJUSDT_5m.csv', parse_dates=True) 

# df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
# # # Convert 'date' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])  # Assuming the 'date' is in milliseconds

# # # Set 'date' as index
df.set_index('datetime', inplace=True)

In [39]:
df["EMA_slow"]=ta.ema(df.close, length=50)
df["EMA_fast"]=ta.ema(df.close, length=30)
df['RSI']=ta.rsi(df.close, length=10)
my_bbands = ta.bbands(df.close, length=15, std=1.5)
df['ATR']=ta.atr(df.high, df.low, df.close, length=7)
df=df.join(my_bbands)
df

,open,high,low,close,volume,quote_volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
datetime,,,,,,,,,,,,,,,
2022-11-21 03:15:00,1.564,1.564,1.559,1.559,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:20:00,1.559,1.560,1.556,1.557,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:25:00,1.557,1.564,1.553,1.563,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:30:00,1.563,1.576,1.563,1.576,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:35:00,1.576,1.577,1.575,1.577,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04 04:05:00,15.645,15.651,15.603,15.608,0,0,15.665729,15.608649,50.878176,0.050655,15.558226,15.610200,15.662174,0.665893,0.478835
2024-09-04 04:10:00,15.608,15.630,15.600,15.627,0,0,15.664211,15.609833,53.960180,0.047704,15.578470,15.617133,15.655797,0.495141,0.627597
2024-09-04 04:15:00,15.627,15.631,15.572,15.576,0,0,15.660751,15.607650,45.454491,0.049318,15.577603,15.616867,15.656131,0.502841,-0.020409


In [40]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-100000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

 10%|█         | 10104/99999 [01:34<13:56, 107.48it/s]


KeyboardInterrupt: 

In [27]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 9999/9999 [00:47<00:00, 209.82it/s]
/tmp/ipykernel_2557104/1508173914.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['low']-1e-3
    elif x['TotalSignal']==1:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

/tmp/ipykernel_2557104/3164771608.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [30]:
def SIGNAL():
    return df.TotalSignal

In [31]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'})
df.set_index('datetime', inplace=True)
bt = Backtest(df, MyStrat, cash=250, margin=1/30)


In [32]:
df

,Open,High,Low,Close,volume,quote_volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal,pointpos
datetime,,,,,,,,,,,,,,,,,,
2024-07-31 11:10:00,22.967,23.068,22.967,23.037,0,0,23.146748,23.105371,45.403112,0.073695,22.958081,23.042733,23.127385,0.734738,0.466136,1,0,NaN
2024-07-31 11:15:00,23.037,23.130,23.037,23.087,0,0,23.144405,23.104186,53.052099,0.076453,22.958403,23.041867,23.125331,0.724456,0.770376,1,0,NaN
2024-07-31 11:20:00,23.087,23.122,23.047,23.053,0,0,23.140821,23.100883,47.973924,0.076246,22.957850,23.040467,23.123084,0.717146,0.575852,1,0,NaN
2024-07-31 11:25:00,23.053,23.072,23.012,23.039,0,0,23.136828,23.096891,45.961116,0.073925,22.956780,23.037267,23.117753,0.698748,0.510768,1,0,NaN
2024-07-31 11:30:00,23.039,23.096,23.038,23.081,0,0,23.134639,23.095866,52.591411,0.071650,22.957444,23.036000,23.114556,0.682026,0.786421,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04 04:00:00,15.606,15.645,15.590,15.645,0,0,15.668086,15.608693,57.640886,0.051098,15.550929,15.606733,15.662538,0.715133,0.842864,1,0,NaN
2024-09-04 04:05:00,15.645,15.651,15.603,15.608,0,0,15.665729,15.608649,50.878176,0.050655,15.558226,15.610200,15.662174,0.665893,0.478835,1,0,NaN
2024-09-04 04:10:00,15.608,15.630,15.600,15.627,0,0,15.664211,15.609833,53.960180,0.047704,15.578470,15.617133,15.655797,0.495141,0.627597,1,0,NaN


In [33]:
bt.run()

Start                     2024-07-31 11:10:00
End                       2024-09-04 04:20:00
Duration                     34 days 17:10:00
Exposure Time [%]                         0.0
Equity Final [$]                        250.0
Equity Peak [$]                         250.0
Return [%]                                0.0
Buy & Hold Return [%]               -32.43912
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [34]:
bt.plot()

ValueError: failed to validate DatetimeTickFormatter(id='p1199', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list